In [1]:
import torch
import torchvision
from torchvision import transforms
from torchvision import datasets 

In [2]:
data_train = datasets.FashionMNIST(root='../data', 
                            transform=transforms.Compose([transforms.ToTensor()]),
                           train=True, download=True)
data_test = datasets.FashionMNIST(root='../data/', 
                            transform=transforms.Compose([transforms.ToTensor()]),
                           train=False, download=True)

In [4]:
data_loader_train = torch.utils.data.DataLoader(dataset=data_train, batch_size=256, shuffle=True)
data_loader_test = torch.utils.data.DataLoader(dataset=data_test, batch_size=256, shuffle=True)

In [5]:
class CNN_Model(torch.nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=5, padding=2),
            
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(stride=2,kernel_size=2))
        
        self.conv2=torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=5),
            
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(stride=2, kernel_size=2))
        
        self.dense = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(16 * 5 * 5, 120),
            torch.nn.Sigmoid(),
            
            torch.nn.Linear(120, 84),
            torch.nn.Sigmoid(),
            torch.nn.Linear(84, 10))
        
    # 前向传播
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        
        x = self.dense(x2)
        return x

In [6]:
# 模型实例化
cnn_model = CNN_Model()

# 构造损失函数
loss_func = torch.nn.CrossEntropyLoss()

learning_rate = 0.001
num_epochs = 10
batch_size = 256


# 构建优化器
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=learning_rate)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

cnn_model.to(device)

CNN_Model(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Sigmoid()
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): Sigmoid()
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [15]:
# 训练过程
for epoch in range(num_epochs):
    running_loss = 0.0
    running_correct = 0.0
    print('Epoch {}/{}'.format(epoch, num_epochs))
    for inputs, labels in data_loader_train:
        inputs = inputs.to(device)
        labels = labels.to(device) 
        
        optimizer.zero_grad()
        
        outputs = cnn_model(inputs)
        
        loss = loss_func(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        
        _,  pred = torch.max(outputs.data, 1)

        running_loss += loss
        running_correct += torch.sum(pred == labels)
    print(f'accuracy is {running_correct / len(data_train)}')

Epoch 0/10
accuracy is 0.3388499915599823
Epoch 1/10
accuracy is 0.6506666541099548
Epoch 2/10
accuracy is 0.7266833186149597
Epoch 3/10
accuracy is 0.750083327293396
Epoch 4/10
accuracy is 0.7719500064849854
Epoch 5/10
accuracy is 0.7887833714485168
Epoch 6/10
accuracy is 0.8055333495140076
Epoch 7/10
accuracy is 0.8179166913032532
Epoch 8/10
accuracy is 0.8249499797821045
Epoch 9/10
accuracy is 0.8323000073432922
